In [28]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve
from tensorflow.models.rnn.translate import seq2seq_model
import time
from os import listdir
from os.path import isfile, join
import codecs,re

In [26]:
sentence_pattern = "[\(|\)|\.|:|\?|;|!|,|\r|\n|\d|\s]+"

dfile = "./data/autobild_1_02_2013_0_decode.txt"
efile = "./data/autobild_1_02_2013_0_encode.txt"
def load_text_data(file_name):
    data = []
    with codecs.open(file_name,"r","latin-1") as f:
        d_text = "".join([l+" " for l in f.readlines()]).lower();
        for s in re.split(sentence_pattern,d_text):
            data.append(s)
    return data
#edata = load_text_data(efile)
#ddata = load_text_data(dfile)

#for e,d in zip(edata, ddata):
#    print(e, "", d)
    
def load_pair_data(file_name):
    data = []
    with codecs.open(file_name,"r","latin-1") as f:
        for l in f:
            data.append(re.split("[\r|\n|\s|\|]+",l)[::2])
    return data
#pair = load_pair_data("./data/autobild_1_02_2013_0_pair.txt")

def load_pair_data_from_dir(dir_path):
    pair = []
    for f in [join(dir_path, f) for f in listdir(dir_path) if isfile(join(dir_path, f))]:
        pair += load_pair_data(f)
    print("data loaded: ", len(pair))
    return pair

pair = load_pair_data_from_dir("/data/korpora/korpora_tagged/")
print(pair[:100])


data loaded:  2857875
[[u'35', u'35'], [u'von', u'von'], [u'200', u'200'], [u'Der', u'd'], [u'mit', u'mit'], [u'dem', u'd'], [u'Hai', u'Hai'], [u'taucht', u'tauchen'], [u';', u';'], [u'Auf', u'auf'], [u'den', u'd'], [u'Malediven', u'Malediven'], [u'kann', u'k\xf6nnen'], [u'man', u'man'], [u'Urlaub', u'Urlaub'], [u'und', u'und'], [u'Umweltschutz', u'Umweltschutz'], [u'gut', u'gut'], [u'miteinander', u'miteinander'], [u'verbinden', u'verbinden'], [u',', u','], [u'findet', u'finden'], [u'Schauspieler', u'Schauspieler'], [u'Hannes', u'Hanne', u''], [u'Jaenicke', u'Jaenicke'], [u'.', u'.'], [u'Hier', u'hier'], [u'beschreibt', u'beschreiben'], [u'er', u'er'], [u'seine', u'sein'], [u'Begegnung', u'Begegnung'], [u'mit', u'mit'], [u'Walhaien', u'Walhaien'], [u',', u','], [u'den', u'd'], [u'gr\xf6\xdften', u'gro\xdf'], [u'Fischen', u'Fisch', u''], [u'der', u'd'], [u'Welt', u'Welt'], [u',', u','], [u'die', u'd'], [u'vom', u'vom'], [u'Aussterben', u'Aussterben'], [u'bedroht', u'bedroht'], [u'sind'

In [29]:
reserved_letters = [' ','#','$','?',u'\xf6',u'\xe4',u'\xdf',u'\xfc',u'\xc4',u'\xd6',u'\xdc','-','.']
reserved_letters_len = len(reserved_letters)
EOF = reserved_letters.index('$')
UNK = reserved_letters.index('?')

vocabulary_size = len(string.ascii_lowercase)+ len(string.ascii_uppercase) + reserved_letters_len
lower_length = len(string.ascii_lowercase)
first_lower_letter = ord(string.ascii_lowercase[0])
first_upper_letter = ord(string.ascii_uppercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_lower_letter + reserved_letters_len
  elif char in string.ascii_uppercase:
    return ord(char) - first_upper_letter + reserved_letters_len + lower_length
  try:
    id = reserved_letters.index(char)
    return id
  except:
    #print('Unexpected character: %s' % char)
    return UNK
  
def id2char(dictid):
  if dictid >= reserved_letters_len and dictid < reserved_letters_len + lower_length:
    return chr(dictid + first_lower_letter - reserved_letters_len)
  elif dictid >= reserved_letters_len + lower_length:
    return chr(dictid + first_upper_letter - reserved_letters_len - lower_length)
  elif dictid >= 0:
    return reserved_letters[dictid]
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'), char2id(u'\xf6'),char2id('Z'))
print(id2char(char2id('a')), id2char(26), id2char(0),id2char(char2id('B')))


13 38 0 3 4 64
a n   B


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [32]:
_buckets = [(10, 10), (15, 15), (25, 25), (50, 50)]
def prepare_data(pair_data):
    data_set = [[] for _ in _buckets]
    for d in pair_data:
        print(d)
        if len(d) >= 2:
            print(d)
            s_id = [char2id(c) for c in d[0]]
            t_id = [char2id(c) for c in d[1]]
            t_id.append(EOF)
            for bucket_id, (source_size, target_size) in enumerate(_buckets):
              if len(s_id) < source_size and len(t_id) < target_size:
                print(s_id)
                if len(s_id)>0 and sum(s_id)/len(s_id) > UNK:
                    data_set[bucket_id].append([s_id, t_id])
                break
    return data_set

print(len(pair), "pairs")
valid_set_size = 10000
valid_set = prepare_data(pair[:valid_set_size])
#train_set = prepare_data(pair[valid_set_size+1:])
for v in valid_set:
    print([["".join([id2char(c) for c in p]) for p in b] for b in v[:50]])

2857875 pairs
[u'35', u'35']
[u'35', u'35']
[3, 3]
[u'von', u'von']
[u'von', u'von']
[34, 27, 26]
[u'200', u'200']
[u'200', u'200']
[3, 3, 3]
[u'Der', u'd']
[u'Der', u'd']
[42, 17, 30]
[u'mit', u'mit']
[u'mit', u'mit']
[25, 21, 32]
[u'dem', u'd']
[u'dem', u'd']
[16, 17, 25]
[u'Hai', u'Hai']
[u'Hai', u'Hai']
[46, 13, 21]
[u'taucht', u'tauchen']
[u'taucht', u'tauchen']
[32, 13, 33, 15, 20, 32]
[u';', u';']
[u';', u';']
[3]
[u'Auf', u'auf']
[u'Auf', u'auf']
[39, 33, 18]
[u'den', u'd']
[u'den', u'd']
[16, 17, 26]
[u'Malediven', u'Malediven']
[u'Malediven', u'Malediven']
[51, 13, 24, 17, 16, 21, 34, 17, 26]
[u'kann', u'k\xf6nnen']
[u'kann', u'k\xf6nnen']
[23, 13, 26, 26]
[u'man', u'man']
[u'man', u'man']
[25, 13, 26]
[u'Urlaub', u'Urlaub']
[u'Urlaub', u'Urlaub']
[59, 30, 24, 13, 33, 14]
[u'und', u'und']
[u'und', u'und']
[33, 26, 16]
[u'Umweltschutz', u'Umweltschutz']
[u'Umweltschutz', u'Umweltschutz']
[59, 25, 35, 17, 24, 32, 31, 15, 20, 33, 32, 38]
[u'gut', u'gut']
[u'gut', u'gut']
[19, 33

In [ ]:
num_nodes = 512
graph = tf.Graph()
batch_size = 64
with graph.as_default():
  model = seq2seq_model.Seq2SeqModel(source_vocab_size = vocabulary_size,
                                       target_vocab_size = vocabulary_size,
                                       buckets = _buckets,
                                       size = num_nodes,
                                       num_layers = 2, # one encoding and one decoding LSTM
                                       max_gradient_norm = 5.0,
                                       batch_size = batch_size,
                                       learning_rate = 1.0,
                                       learning_rate_decay_factor = 0.9,
                                       use_lstm = True,
                                       forward_only = False)



In [45]:
def evaluate_valid(bucket_id, printed_size):
    valid_batches = model.get_batch(valid_set, bucket_id) 
    _, eval_loss, output_logits = model.step(session, valid_batches[0], valid_batches[1],
                                       valid_batches[2], bucket_id, True)
    eval_ppx = np.exp(float(eval_loss)) if eval_loss < 300 else float(
              "inf")
    print("  eval perplexity: %.2f" % (eval_ppx))
    valid_input = np.transpose(valid_batches[0])
    valid_decode = np.transpose(valid_batches[1])
    for i in range(min(printed_size, len(valid_input))):
      outputs = [int(np.argmax(logit[i:i+1], axis=1)) for logit in output_logits]
      if EOF in outputs:
        outputs = outputs[0:outputs.index(EOF)]          
      print("  sampled valid (i,t,o)", "".join([id2char(o) for o in valid_input[i]])[::-1],"".join([id2char(o) for o in valid_decode[i]]),"".join([id2char(o) for o in outputs]))


In [54]:
num_steps = 20000
summary_frequency = 100
checkpoint_frequency = 1000
checkpoint_path = os.path.join("/data/checkpoints", "wordform.ckpt")

train_bucket_sizes = [len(train_set[b]) for b in xrange(len(_buckets))]
train_total_size = float(sum(train_bucket_sizes))
train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size
                           for i in xrange(len(train_bucket_sizes))]


with tf.Session(graph=graph) as session:
  step_time = 0
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  previous_losses = []
  for step in range(num_steps):
    random_number_01 = np.random.random_sample()
    bucket_id = min([i for i in xrange(len(train_buckets_scale))
                       if train_buckets_scale[i] > random_number_01])
    
    e,d,w = model.get_batch(train_set, bucket_id)
    start_time = time.time()
    n,l,_ = model.step(session=session, encoder_inputs=e,decoder_inputs=d,target_weights=w,bucket_id=bucket_id,forward_only=False)
    mean_loss += l/summary_frequency
    step_time += (time.time() - start_time) / summary_frequency
    if (step+1) % checkpoint_frequency == 0:
      print("checkpoint...")
      model.saver.save(session, checkpoint_path, global_step=model.global_step)
    
    if (step+1) % summary_frequency == 0:
      # The mean loss is an estimate of the loss over the last few batches.
      perplexity = np.exp(float(mean_loss)) if mean_loss < 300 else float("inf")
      print ("global step %d learning rate %.4f step-time %.2f train perplexity "
               "%.2f" % (model.global_step.eval(), model.learning_rate.eval(),
                         step_time, perplexity))
      if len(previous_losses) > 2 and mean_loss > max(previous_losses[-3:]):
          print("run learning rate decay...")
          session.run(model.learning_rate_decay_op)
      previous_losses.append(mean_loss)
      mean_loss = 0
      step_time = 0
    
      evaluate_valid(bucket_id,20)
  print("Done.") 
  for b in xrange(len(_buckets)):
    evaluate_valid(b,batch_size)

Initialized
global step 100 learning rate 1.0000 step-time 1.04 train perplexity 19.09
  eval perplexity: 5.80
  sampled valid (i,t,o) der        #d.        d
  sampled valid (i,t,o) der        #d.        d
  sampled valid (i,t,o) alles      #alle.     llll
  sampled valid (i,t,o) weiß       #weiß.     ee
  sampled valid (i,t,o) das        #d.        d
  sampled valid (i,t,o) das        #d.        d
  sampled valid (i,t,o) wie        #wie.      d
  sampled valid (i,t,o) ist        #sein.     si
  sampled valid (i,t,o) das        #d.        d
  sampled valid (i,t,o) al-Duri    #al-Duri.  aaaa
  sampled valid (i,t,o) An         #an.       n
  sampled valid (i,t,o) war        #sein.     a
  sampled valid (i,t,o) ist        #sein.     si
  sampled valid (i,t,o) auf        #auf.      aa
  sampled valid (i,t,o) Uhr        #Uhr.      
  sampled valid (i,t,o) York       #York.     oor
  sampled valid (i,t,o) W.         #W..       
  sampled valid (i,t,o) records    #records.  rrrrrrrrrr
  samp